In [ ]:
import sys
import torch
import os
import locale # sometimes needed to fix errors in Colab
locale.getpreferredencoding = lambda: "UTF-8"

HIPP-NN or hippynn (Hierarchical Interacting-Particle Neural Network) is a backend for training machine learning models for molecular predictions. It integrates with:
- [PYSEQM](https://https://github.com/lanl/PYSEQM) for building differntiable Hamiltonians
- [LAMMPS](https://www.lammps.org/) for large-scale simulations
- [ASE](https://wiki.fysik.dtu.dk/ase/) as an external calculator for static and dynamic calculations
- [ALF](https://github.com/lanl/ALF/tree/main) for running active learning - collecting data from most underrepsented regions to improve accuracy

![HIPNN](https://raw.githubusercontent.com/NikitaFedik/mlchem_ndsu2023/tree/main/graphics/hippn_variants.jpg)


💡 Jupyter Notebook (and Google Colab) can execute Linux commands.  Very handy for installing packages.    
⚠ Installed packages are not permanently saved in Google Colab. There are workarounds - [see Medium](https://netraneupane.medium.com/how-to-install-libraries-permanently-in-google-colab-fb15a585d8a5).  
<br/>

To install HIPPYNN from [github](https://github.com/lanl/hippynn) run the command:

In [ ]:
!pip install git+https://github.com/lanl/hippynn

In [ ]:
!git clone https://github.com/NikitaFedik/mlchem_ndsu2023.git

In [ ]:
import hippynn

In [ ]:
!pwd

In [ ]:
# Check that 20k_ANI1x exists and contains all needed arrays
!ls mlchem_ndsu2023/data

### Hyperparameters for the network

Selection of hyperparameters is based on both intuition and hyperparam search (**random search** or more extensive and expensive **grid search**).

💡 Most importantly, tune `n_features` and `n_sensitivities` for optimal performace.

<br/>

| Parameter             | Value              | Explanation                                                      | Tips                                                        |
|-----------------------|--------------------|------------------------------------------------------------------|-------------------------------------------------------------|
| possible_species      | [0, 1, 6, 7, 8] | Z values of the elements in ANI1-x                               | Always add 0 and sort in asceinding order          |
| n_features            | 60                 | Number of neurons at each layer                                 | Start with small values like 40, typically ~128  |
| n_sensitivities       | 20                 | Number of sensitivity functions in an interaction layer          | Typically, 30-40                |
| dist_soft_min         | 0.8                | Minimum distance (in Å) for atomic pairs        | Choose an appropriate value for your specific application.   |
| dist_soft_max         | 5.0               | Spatial cut-off (in Å)         | Adjust based on the expected interaction range.             |
| dist_hard_max         | 6.5               | Maximum distance (in Å) | Ensure it's slightly greater than dist_soft_max for effective cutoffs. |
| n_interaction_layers  | 2                  | Number of interaction blocks                                     | 1 = neigbours only, 2 = neighbours of neighbours; 2 is optimal for molecules     |
| n_atom_layers         | 3                  | Number of linear layers                    | 3 is usually an optimal value           |


In [ ]:
netname = "TEST" # name of the folder for your network

In [ ]:
network_params = {
    "possible_species": [0, 1, 6, 7, 8],  # Z values of the elements
    "n_features": 80,                     # Number of neurons at each layer
    "n_sensitivities": 30,                # Number of sensitivity functions at interaction layer
    "dist_soft_min": 0.8,                 # Angstrom
    "dist_soft_max": 5.0,
    "dist_hard_max": 6.5,
    "n_interaction_layers": 2,            # Number of interaction blocks
    "n_atom_layers": 3,                   # Number of atom layers in an interaction block
}

In [ ]:
hippynn.settings.WARN_LOW_DISTANCES=False  # supress lots of printing

In [ ]:
# Define a model
import hippynn
from hippynn.graphs import inputs, networks, targets, physics


species = inputs.SpeciesNode(db_name="Z")      # name for array with atomic numbers
positions = inputs.PositionsNode(db_name="R")  # xyz corrdinates as

network = networks.Hipnn("hipnn_model", (species, positions), module_kwargs=network_params)
henergy = targets.HEnergyNode("HEnergy", network, db_name="E")                        #
force   = physics.GradientNode("forces", (henergy, positions), db_name = 'F', sign=1) # atomic forces


hierarchicality = henergy.hierarchicality  # hiearchical ansatz

# define loss quantities - function that meausers how close predictions are to true values

from hippynn.graphs.nodes.loss import MSELoss, MAELoss, Rsq, Mean, l2reg
losses = {
    "E-RMSE": MSELoss.of_node(henergy) ** (1/2), # root
    "E-MAE" : MAELoss.of_node(henergy),
    "F-RMSE": MSELoss.of_node(force) ** (1/2), #root
    "F-MAE" : MAELoss.of_node(force),
    "l2_reg": l2reg(network),
    "rbar"  : Mean.of_node(hierarchicality),

}

losses["EnergyTotal"]     = losses['E-RMSE']        + losses["E-MAE"]
losses["ForceTotal"]      = losses["F-RMSE"]        + losses["F-MAE"]
losses["E_F_LossTotal"]   = losses["EnergyTotal"]  # + losses["ForceTotal"] * 0.4 # !!! IMPORTANT
                                                                            # you should tune/think of coeff here
# Final train loss :
losses["LossTotal"]       = losses["E_F_LossTotal"] +  1.0e-3 * losses["l2_reg"] + losses["rbar"]

# Validation losses are what we check on the data between epochs -- we can only train to
# a single loss, but we can check other metrics too to better understand how the model is training.
# There will also be plots of these things over time when training completes.


In [ ]:
# This piece of code glues the stuff together as a pytorch model,
# dropping things that are irrelevant for the losses defined.
training_modules, db_info = hippynn.experiment.assemble_for_training(losses["LossTotal"], losses)

In [ ]:
# Go to a directory for the model.
# hippynn will save training files in the current working directory.
from hippynn.experiment.controllers import RaiseBatchSizeOnPlateau, PatienceController

with hippynn.tools.active_directory(netname):
    # Log the output of python to `training_log.txt`
    with hippynn.tools.log_terminal("training_log.txt", "wt"):
        database = hippynn.databases.DirectoryDatabase(
            name="",  # Prefix for arrays in the directory
            directory="/content/mlchem_ndsu2023/data",
            test_size=0.1,   # Fraction or number of samples to test on
            valid_size=0.1,  # Fraction or number of samples to validate on
            seed=2001,       # Random seed for splitting data
            **db_info,       # Adds the inputs and targets db_names from the model as things to load
        )

        # Now that we have a database and a model, we can
        # Fit the non-interacting energies by examining the database.
        # This tends to stabilize training a lot.
        from hippynn.pretraining import set_e0_values

        set_e0_values(henergy, database, trainable_after=True) # Substract energy of atoms
                                                               # This will use formation energies

        # Parameters describing the training procedure.
        from hippynn.experiment import setup_and_train

        experiment_params = hippynn.experiment.SetupParams(
            stopping_key="E-RMSE",  # The name in the validation_losses dictionary.
            batch_size = 1024,           # how many data entries consider at once - bound by GPU memory
            learning_rate = 0.001,      # weight of gradient
            optimizer=torch.optim.Adam, # intelligently adapts learning rate
            max_epochs = 60,            # how many times to iterate through data
            )

        setup_and_train(
            training_modules = training_modules,
            database = database,
            setup_params = experiment_params,
        )

Carefull examine reported accuracy. You can always check `training_log.txt`.
Most likely, errors for valid and test are **higher** than for train test.
- 💡 Inspect your plots in folder `over_time` to get insights on how the accuracy changes over time (`E-RMSE.pdf`, `F-RMSE.pdf` etc)


This could be an examle of overfitting, and this is epxected for small datasets.
- 💡 First thing to try is to collect more data. Typically, interatomic potentials include 500k-20 million points while we used only 50k.
- 💡 Deploy active learning (less is more) - collect data from underrepresented regions. Stay tuned for docs and full release of ALF.

### HIPNN for property prediction

We will consider how to use HIP-NN to make predictions on energy and atomic forces. Those predictions are good on their own as single-point calculations or could be couple with molecular dynamics drivers through ASE.

In [ ]:
! pwd # check where you are - you need to do os.chdir(folder with model)

In [ ]:
# go to the directory with the model
os.chdir(netname)

In [ ]:
!pwd

In [ ]:
!ls

In [ ]:
# load model
from hippynn.experiment.serialization import load_model_from_cwd
model = load_model_from_cwd()

In [ ]:
from pathlib import Path
from natsort import natsorted

def list_files(path, ext):
    """list files in a directory with a given extension
    Args:
        path (): _description_
        ext (str): extension without the dot (log, out, ...)

    Returns:
        list: collection of files with the given extension
    """
    path = Path(path)
    files =  [str(x) for x in path.iterdir() if x.is_file() and x.suffix == f'.{ext}']
    files = natsorted(files) # natsort by filenames; pathlib objects could not be sorted directly!

    return files

In [ ]:
!ls

In [ ]:
xyz_files = list_files('/content/mlchem_ndsu2023/data/azo_rotation', 'xyz')

In [ ]:
xyz_files[:20] # print only 20 to inspect

In [ ]:
# parse the data for rotational profile - this one of the paths, not the best possible
from ase.io import read as aseread
from ase.units import eV, kcal, mol
atoms_list = [aseread(x) for x in xyz_files] # read xyz files and create atoms objects of ASE


In [ ]:
R_azo = [x.positions for x in atoms_list] # get coordinates
R_azo = torch.Tensor(R_azo).to('cuda')

In [ ]:
Z_azo = [x.numbers for x in atoms_list] # get coordinates
Z_azo = torch.Tensor(Z_azo).to('cuda').long()

In [ ]:
# create a predictor - will estimate properties on which model was trained

predictor = hippynn.graphs.Predictor.from_graph(model)

In [ ]:
outputs = predictor(Z = Z_azo, R = R_azo)

In [ ]:
outputs.keys()

In [ ]:
ML_E = outputs['E'] # tensor of ML energies

In [ ]:
ML_E_shifted = [(x - ML_E[0]) for x in ML_E] # use relative eneregies, not absolute

In [ ]:
!ls

In [ ]:
import numpy as np
QM_E = np.load('/content/mlchem_ndsu2023/data/azo_rotation/QM_E_azo_refined_path.npy')

In [ ]:
# plot the data to compare HIPNN potential vs reference DFT
# it is okay if agreement is not great - we used small fraction of data and trained "toy" model

import matplotlib.ticker as ticker
from matplotlib import pyplot as plt
formatter = ticker.FormatStrFormatter('%.0f')

fig, ax1 = plt.subplots()
ax2 = ax2 = ax1.twinx()

ax1.set_ylabel('E, kcal/mol')
ax1.set_xlabel('reaction coordinate')
ax1.plot(QM_E, label = 'DFT closed-shell', color = 'r')   # plot DFT data
ax1.plot(ML_E_shifted, label = 'hippynn', color = 'grey') # plot ML data
ax1.legend(loc='upper left')

plt.savefig('azo_pure_rotational_profile.png', dpi=300)

Do not be dissapointed - ML can reproduce rotational profiles pretty well!   
More data/longer/training/more complex is needed. "Chemical accuracy" of 1 kcal/mol for a single molecule are reachable


